In [1]:
from src.linear_causality import LinearLKInformationFlow

In [2]:
import numpy as np
from tqdm import tqdm
# from causality_subspace import causality_subspace

dt=1
pn=1
T1=[];T2=[]
# pca = PCA(n_components=5)  # 假设我们想将数据降到2维  
Nt=10000;
np.random.seed = 5
# for i in tqdm(range(100)):
X1=np.random.randn(5,Nt)*0.2
X2=np.random.randn(5,Nt)*0.2

A1=np.array([[0.9,0.6,0,0.6,0],[0,0.8,0.6,0,0.6],[0,0,0.9,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T;
A2=np.array([[0.2,0.6,0,0.6,0],[0,0.2,0.6,0,0.6],[0,0,0.2,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T;

B1=-np.array([[0,0,0,0,0],[0,0.,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]);
B2=-np.array([[0,0,0,0,0],[0,0.0,0,0,0],[0,0,0,0,0],[0,0,0,0.4,0],[0,0,0,0,0]]);
for it in range(1,Nt):
    random_noise_1 = np.random.randn(5)*0.1
    random_noise_2 = np.random.randn(5)*0.1
    X1[:,it]=A1@X1[:,it-1]+X1[:,it]+1*B1@X2[:,it-1] + random_noise_1
    X2[:,it]=A2@X2[:,it-1]+X2[:,it]+1*B2@X1[:,it-1] + random_noise_2
    
XX=np.zeros([10,Nt])
XX[:5]=X1
XX[5:]=X2

In [3]:
A1

array([[0.9, 0. , 0. , 0. , 0. ],
       [0.6, 0.8, 0. , 0. , 0. ],
       [0. , 0.6, 0.9, 0. , 0. ],
       [0.6, 0. , 0. , 0.2, 0. ],
       [0. , 0.6, 0. , 0. , 0.2]])

In [38]:
# segments = [(0, 1),(1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,8),(8,9),(9,10)]
segments = [(0, 5),(5,10)]

In [39]:
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(XX[:,-10000:].T, lag_list=[1,2,5], segments=segments)

In [40]:
a = lkif.information_flow[:,:10][:2,:2]

In [41]:
b = (lkif.information_flow[:,:10]/lkif.normalizer)[:2,:2]

In [42]:
lkif.information_flow

array([[-1.98587680e+00, -2.20608196e-04, -6.06783741e-04,
         3.46907244e-04, -1.67876576e-03, -3.23286078e-04],
       [ 3.55277492e-01, -4.01073145e+00, -9.98656920e-02,
         1.58709110e-03,  6.59199406e-03,  1.38764531e-03]])

In [43]:
result_dict = lkif.get_dict()

In [44]:
result_dict["information_flow"]

array([[-1.98587680e+00, -2.20608196e-04, -6.06783741e-04,
         3.46907244e-04, -1.67876576e-03, -3.23286078e-04],
       [ 3.55277492e-01, -4.01073145e+00, -9.98656920e-02,
         1.58709110e-03,  6.59199406e-03,  1.38764531e-03]])

In [45]:
result_dict["statistics"]["p99_critical_value"]

array([[0.0576074 , 0.00451078, 0.04896584, 0.00291211, 0.02481565,
        0.00233536],
       [0.19446882, 0.05760585, 0.21145913, 0.02738143, 0.08219894,
        0.00500308]])

In [46]:
result_dict["statistics"]["p95_critical_value"]

array([[0.04383382, 0.00343228, 0.0372584 , 0.00221584, 0.01888237,
        0.00177699],
       [0.14797249, 0.04383264, 0.16090052, 0.02083469, 0.06254566,
        0.00380687]])

In [48]:
np.abs(result_dict["information_flow"]) > result_dict["statistics"]["p99_critical_value"]

array([[ True, False, False, False, False, False],
       [ True,  True, False, False, False, False]])